# Split the basin example base model

## Notebook Setup

In [ ]:
import os
import sys
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pathlib as pl
from shapely.geometry import Polygon, LineString

import flopy
from flopy.discretization import StructuredGrid
from flopy.mf6.utils import Mf6Splitter
import flopy.plot.styles as styles

In [ ]:
# import all plot style information from defaults.py
from defaults import *

### Parallel settings

1. Set the number of models in the row and column directions. NOTE: Set nrow_blocks or ncol_blocks to 0 to use metis.

In [ ]:
nrow_blocks, ncol_blocks = 8, 0

In [ ]:
parallel_ws = get_parallel_data(nrow_blocks, ncol_blocks)
parallel_ws

### Load the base basin model 

In [ ]:
base_ws = "../examples/basin_base"

In [ ]:
base_sim = flopy.mf6.MFSimulation.load(sim_name="basin", sim_ws=base_ws)

In [ ]:
base_gwf = base_sim.get_model()

### Load the parallel basin model

In [ ]:
new_sim = flopy.mf6.MFSimulation.load(sim_name="basin", sim_ws=parallel_ws)

#### Load the node mapping array for the parallel basin model

In [ ]:
json_path = parallel_ws / "mfsplit_node_mapping.json"
mfsplit = Mf6Splitter(base_sim)
mfsplit.load_node_mapping(json_path)

### Plot the multi-model and single model heads

In [ ]:
base_head = base_gwf.output.head().get_data()
hmin, hmax = (
    base_head.min(),
    np.where(base_head < 1e30, base_head, 0).max(),
)
contours = np.arange(0, 100, 10)

#### Build a dictionary with the model heads for each partition

In [ ]:
mfsplit.

In [ ]:
model_names = list(new_sim.model_names)
head_dict = {}
for modelname in model_names:
    mnum = int(modelname.split("_")[-1])
    head = new_sim.get_model(modelname).output.head().get_alldata()[-1]
    head_dict[mnum] = head

#### Build a single head array

In [ ]:
new_head = mfsplit.reconstruct_array(head_dict)

#### Plot the results

In [ ]:
with styles.USGSMap():
    fig = plt.figure(figsize=(figwidth, figheight * 1.3333))
    t = new_head - base_gwf
    hv = [new_head, gwfhead_base, t]
    titles = ["Multiple models", "Single model", "Multiple - single"]
    for idx in range(3):
        ax = fig.add_subplot(3, 1, idx + 1)
        ax.set_aspect("equal")
        ax.set_title(titles[idx])

        if idx < 2:
            levels = contours
            vmin = hmin
            vmax = hmax
        else:
            levels = None
            vmin = None
            vmax = None

        pmv = flopy.plot.PlotMapView(model=gwf_base, ax=ax, layer=0)
        h = pmv.plot_array(hv[idx], vmin=vmin, vmax=vmax)
        if levels is not None:
            c = pmv.contour_array(
                hv[idx],
                levels=levels,
                colors="white",
                linewidths=0.75,
                linestyles=":",
            )
            plt.clabel(c, fontsize=8)
        pmv.plot_inactive()
        plt.colorbar(h, ax=ax, shrink=0.5)